In [302]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [303]:
NVH_metrics = ['6208 Cage Failure', 'CH/Input Bearing Race Defect', 'CH/Input Bearing Ball Defect',
						'CH Inter Bearing Race Defect',
						'CH Inter Bearing Ball Defect',
						'Input Gear Clicking',
						'Intermediate Gear Clicking',
						'Output Gear Clicking',
						'Intermediate Gear Clicking Negative',
						'Intermediate Gear Clicking Positive',
						'Input Gear Clicking Positive',
						'Input Gear Clicking Negative',
						'Output Gear Clicking Negative',
						'Output Gear Clicking Positive', 
                        'Motor 3rd Ord Avg', 
                        'Growl 4-6']

In [304]:
df_raw = pd.read_csv('datasets\Week25.csv')
df_raw.head()

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek
0,254065,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,32:52.0,6,17,2024,5,32,6208 Cage Failure,1,57.046379,25
1,254065,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,32:52.0,6,17,2024,5,32,Max,1,556.361755,25
2,254065,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,32:52.0,6,17,2024,5,32,Torque Inverter Side,1,320.860748,25
3,254066,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,34:33.0,6,17,2024,5,34,Motor 1st Ord Max,1,0.221517,25
4,254066,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,34:33.0,6,17,2024,5,34,Motor 2nd Order Max,1,0.161071,25


In [305]:
summary = pd.Series({'FirstPasses':0,
                     'DLE':0,
                           'Retest Success':0,
                           'Reworked Pass':0,
                           'Pending Rework / Unsuccessful Rework':0,
                           'Machine Health Failure':0
                           })

In [306]:
df = df_raw.copy()
#df.groupby('FX_Thing')['metricispass'].sum().min()
#if a test has 46 metricispass, it's a first time pass

#dropping Golden units
golden_units = ['P1025598-00-U:ST13D0013400', 'P1025598-00-U:ST17A1021978']
df = df[~df['FX_Thing'].isin(golden_units)]

retestTime = 30 #threshold time


In [307]:
#key and datetime
df['thing+metricName'] = df['FX_Thing']+'+'+df['metricName']
df['testTime'] = pd.to_datetime({'year':df['testYear'], 'month':df['testMonth'], 'day':df['testDay'], 'hour':df['testHour'], 'minute':df['testMinute']}) #proper formatting of datetime

In [308]:
df['part'].nunique()

101

In [309]:
#Finding all the first time passes. Every unit with 46 metricispass sum is a first time pass
sum_metricispass = df.groupby('FX_Thing')['metricispass'].sum()
firstPasses = sum_metricispass[sum_metricispass == 46].reset_index()['FX_Thing']
summary['FirstPasses'] = len(firstPasses)

In [310]:
#getting all the failures and the unique failing serials
df_allFailures = df[df['metricispass'] == 0].copy()
df_allFailures.head()

#count of unique failures
failing_serials = df_allFailures['FX_Thing'].drop_duplicates()
len(failing_serials)



10

In [311]:
#anything with only passes, but doesn't have 46 runs, is a DLE
DLEs = df['FX_Thing'].nunique() - len(firstPasses) - len(failing_serials)
summary['DLE'] = DLEs

In [312]:
#DLEs
uniqueSerials = df.drop_duplicates(subset = 'FX_Thing')
uniqueSerials[(~uniqueSerials['FX_Thing'].isin(failing_serials)) & (~uniqueSerials['FX_Thing'].isin(firstPasses))]

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek,thing+metricName
0,254065,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-17 05:32:00,6,17,2024,5,32,6208 Cage Failure,1,57.046379,25,P1025598-01-U:ST14J0065840R2+6208 Cage Failure
1380,254182,1,P1025598-01-U-ST17F1035173,P1025598-01-U:ST17F1035173,1025598-01-U,2024-06-18 08:09:00,6,18,2024,8,9,6208 Cage Failure,1,56.066120,25,P1025598-01-U:ST17F1035173+6208 Cage Failure
3653,254404,1,P1025598-00-U-ST13D0013846,P1025598-00-U:ST13D0013846,1025598-00-U,2024-06-20 06:15:00,6,20,2024,6,15,Motor 1st Ord Max,1,0.166461,25,P1025598-00-U:ST13D0013846+Motor 1st Ord Max
3834,254415,1,P1025598-00-U-ST16A0117537R1,P1025598-00-U:ST16A0117537R1,1025598-00-U,2024-06-20 07:20:00,6,20,2024,7,20,6208 Cage Failure,1,54.048317,25,P1025598-00-U:ST16A0117537R1+6208 Cage Failure
4306,254455,1,P1025598-00-U-ST14H0055310R1,P1025598-00-U:ST14H0055310R1,1025598-00-U,2024-06-20 13:53:00,6,20,2024,13,53,6208 Cage Failure,1,53.727848,25,P1025598-00-U:ST14H0055310R1+6208 Cage Failure
5640,254547,1,P1025598-00-U-ST13D0013858,P1025598-00-U:ST13D0013858,1025598-00-U,2024-06-21 11:31:00,6,21,2024,11,31,Motor 1st Ord Max,1,0.144306,25,P1025598-00-U:ST13D0013858+Motor 1st Ord Max


In [313]:
#getting all the failing and passing tests of the failing thing+metricName combinations
df_failures_and_Passes = df[df['thing+metricName'].isin(df_allFailures['thing+metricName'].unique())]

In [314]:
failures_and_passes_summary = df_failures_and_Passes.drop_duplicates(['part', 'metricName'])[['part', 'metricName', 'thing+metricName']]
failures_and_passes_summary 

,part,metricName,thing+metricName
306,P1025598-01-U-ST15J0109167,Stator Temp Delta,P1025598-01-U:ST15J0109167+Stator Temp Delta
460,P1025598-00-U-ST14I0058850,6208 Cage Failure,P1025598-00-U:ST14I0058850+6208 Cage Failure
481,P1025598-00-U-ST14I0058850,A1 243 Tswp Coast Grms,P1025598-00-U:ST14I0058850+A1 243 Tswp Coast Grms
482,P1025598-00-U-ST14I0058850,A1 243 Tswp Drive Grms-600Nm,P1025598-00-U:ST14I0058850+A1 243 Tswp Drive G...
1800,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking,P1025598-01-U:ST20F1087378+Intermediate Gear C...
1807,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking Negative,P1025598-01-U:ST20F1087378+Intermediate Gear C...
2241,P1025598-00-U-ST16I0124320,CH Inter Bearing Race Defect,P1025598-00-U:ST16I0124320+CH Inter Bearing Ra...
2858,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking,P1025598-00-U:ST13B0009697+Intermediate Gear C...
2865,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0009697+Intermediate Gear C...
2961,P1025598-00-U-ST14L0073039,Output Gear Clicking Negative,P1025598-00-U:ST14L0073039+Output Gear Clickin...


# Is there a passing test?


In [315]:
#checks to see if a passing test exists
def isPassTest(part):
    df_tmp = df_failures_and_Passes[(df_failures_and_Passes['part'] == part) & (df_failures_and_Passes['isTestPass'] == 1)]
    if df_tmp.shape[0] == 0:
        return False
    else:
        return True

In [316]:
failures_and_passes_summary['PassingTest?'] = failures_and_passes_summary['part'].apply(isPassTest)

In [317]:
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?
306,P1025598-01-U-ST15J0109167,Stator Temp Delta,P1025598-01-U:ST15J0109167+Stator Temp Delta,False
460,P1025598-00-U-ST14I0058850,6208 Cage Failure,P1025598-00-U:ST14I0058850+6208 Cage Failure,True
481,P1025598-00-U-ST14I0058850,A1 243 Tswp Coast Grms,P1025598-00-U:ST14I0058850+A1 243 Tswp Coast Grms,True
482,P1025598-00-U-ST14I0058850,A1 243 Tswp Drive Grms-600Nm,P1025598-00-U:ST14I0058850+A1 243 Tswp Drive G...,True
1800,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking,P1025598-01-U:ST20F1087378+Intermediate Gear C...,False
1807,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking Negative,P1025598-01-U:ST20F1087378+Intermediate Gear C...,False
2241,P1025598-00-U-ST16I0124320,CH Inter Bearing Race Defect,P1025598-00-U:ST16I0124320+CH Inter Bearing Ra...,True
2858,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking,P1025598-00-U:ST13B0009697+Intermediate Gear C...,False
2865,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0009697+Intermediate Gear C...,False
2961,P1025598-00-U-ST14L0073039,Output Gear Clicking Negative,P1025598-00-U:ST14L0073039+Output Gear Clickin...,False


In [318]:
def calculateTime(part): #If there is a successful rework, find the time between the latest success and the latest failure
    df_test = df_failures_and_Passes[df_failures_and_Passes['part'] == part].drop_duplicates(['testID', 'isTestPass']).reset_index()
    if 1 in df_test['isTestPass'].values: #if there is a passing value
        passingLoc = df_test[df_test['isTestPass'] == 1].index.max() #max is to find the latest pass
        failingLoc = df_test[df_test['isTestPass'] == 0].index.max() #finds the latest fail
        comparison_result = df_test.loc[passingLoc, 'testTime'] - df_test.loc[failingLoc, 'testTime'] #compares the latest pass with the latest fail to find the difference between
        return comparison_result
    else: return np.nan

In [319]:
def failDisposition(row):
    if row['MachineHealth?'] == True:
        return 'Machine Health Failure'
    elif row['retestSuccess?'] == True:
        return 'Retest Success'
    elif row['PassingTest?'] == True:
        return 'Reworked Pass'
    else:
        return 'Pending Rework / Unsuccessful Rework'
    

In [327]:
#cleaningTable
failures_and_passes_summary['TimeToPass'] = failures_and_passes_summary['part'].apply(calculateTime)
failures_and_passes_summary['TimeToPass_minutes'] = failures_and_passes_summary['TimeToPass'].apply(lambda x: x.total_seconds()/60)
failures_and_passes_summary['retestSuccess?'] = failures_and_passes_summary['TimeToPass_minutes'].apply(lambda x: True if x < retestTime else False)
failures_and_passes_summary['MachineHealth?'] = failures_and_passes_summary['metricName'].apply(lambda x: False if x in NVH_metrics else True)

#create a column to disposition each part based on passingTest? retestSuccess? and machine Health?
failures_and_passes_summary['disposition'] = failures_and_passes_summary.apply(failDisposition, axis = 1)

In [322]:
dispositionCounts = failures_and_passes_summary.drop_duplicates(subset = 'part')['disposition'].value_counts()

In [323]:
for index, values in dispositionCounts.iteritems():
    summary[index] = values

C:\Users\Edmund\AppData\Local\Temp\ipykernel_12536\3669781417.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in dispositionCounts.iteritems():


In [324]:
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?,TimeToPass,TimeToPass_minutes,retestSuccess?,MachineHealth?,disposition
306,P1025598-01-U-ST15J0109167,Stator Temp Delta,P1025598-01-U:ST15J0109167+Stator Temp Delta,False,NaT,NaN,False,True,Machine Health Failure
460,P1025598-00-U-ST14I0058850,6208 Cage Failure,P1025598-00-U:ST14I0058850+6208 Cage Failure,True,1 days 00:21:00,1461.0,False,False,Reworked Pass
481,P1025598-00-U-ST14I0058850,A1 243 Tswp Coast Grms,P1025598-00-U:ST14I0058850+A1 243 Tswp Coast Grms,True,1 days 00:21:00,1461.0,False,True,Machine Health Failure
482,P1025598-00-U-ST14I0058850,A1 243 Tswp Drive Grms-600Nm,P1025598-00-U:ST14I0058850+A1 243 Tswp Drive G...,True,1 days 00:21:00,1461.0,False,True,Machine Health Failure
1800,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking,P1025598-01-U:ST20F1087378+Intermediate Gear C...,False,NaT,NaN,False,False,Pending Rework / Unsuccessful Rework
1807,P1025598-01-U-ST20F1087378,Intermediate Gear Clicking Negative,P1025598-01-U:ST20F1087378+Intermediate Gear C...,False,NaT,NaN,False,False,Pending Rework / Unsuccessful Rework
2241,P1025598-00-U-ST16I0124320,CH Inter Bearing Race Defect,P1025598-00-U:ST16I0124320+CH Inter Bearing Ra...,True,0 days 00:24:00,24.0,True,False,Retest Success
2858,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking,P1025598-00-U:ST13B0009697+Intermediate Gear C...,False,NaT,NaN,False,False,Pending Rework / Unsuccessful Rework
2865,P1025598-00-U-ST13B0009697,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0009697+Intermediate Gear C...,False,NaT,NaN,False,False,Pending Rework / Unsuccessful Rework
2961,P1025598-00-U-ST14L0073039,Output Gear Clicking Negative,P1025598-00-U:ST14L0073039+Output Gear Clickin...,False,NaT,NaN,False,False,Pending Rework / Unsuccessful Rework


In [329]:
#lookup
df_failures_and_Passes[df_failures_and_Passes['part'] == 'P1025598-01-U-ST18K1062403']

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek,thing+metricName
5180,254517,0,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 10:21:00,6,21,2024,10,21,6208 Cage Failure,0,59.395386,25,P1025598-01-U:ST18K1062403+6208 Cage Failure
5226,254520,0,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 10:28:00,6,21,2024,10,28,6208 Cage Failure,0,60.000362,25,P1025598-01-U:ST18K1062403+6208 Cage Failure
5410,254532,0,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 10:58:00,6,21,2024,10,58,6208 Cage Failure,0,59.576973,25,P1025598-01-U:ST18K1062403+6208 Cage Failure
5456,254535,0,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 11:04:00,6,21,2024,11,4,6208 Cage Failure,0,59.539703,25,P1025598-01-U:ST18K1062403+6208 Cage Failure
5502,254538,0,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 11:10:00,6,21,2024,11,10,6208 Cage Failure,0,59.432060,25,P1025598-01-U:ST18K1062403+6208 Cage Failure
5821,254558,1,P1025598-01-U-ST18K1062403,P1025598-01-U:ST18K1062403,1025598-01-U,2024-06-21 12:42:00,6,21,2024,12,42,6208 Cage Failure,1,56.369293,25,P1025598-01-U:ST18K1062403+6208 Cage Failure


In [333]:
summary['firstPassTotal'] = summary['FirstPasses'] + summary['DLE'] + summary['Retest Success'] + summary['Machine Health Failure']
summary['fallout'] = summary['Reworked Pass'] + summary['Pending Rework / Unsuccessful Rework']
summary['FPY'] = summary['firstPassTotal'] / (summary['firstPassTotal'] + summary['fallout'])
summary

FirstPasses                             85.000000
DLE                                      6.000000
Retest Success                           2.000000
Reworked Pass                            3.000000
Pending Rework / Unsuccessful Rework     4.000000
Machine Health Failure                   1.000000
firstPassTotal                          94.000000
fallout                                  7.000000
FPY                                      0.930693
dtype: float64

In [335]:
#output
failures_and_passes_summary.to_csv('summary.csv')
summary.to_csv('summary.csv', mode = 'a')